In [ ]:
import pandas as pd
import tqdm

from ukbb_recessive.data_collection.variants import VariantFeatures
import glob
import os
import datetime
import multiprocessing as mp
import pandas as pd


# Select rare synonymous variants

In [ ]:
path = '.../450k/plp_selection'

In [ ]:
for i in tqdm.tqdm(range (1, 23)):
    input_file = f"{path}/chr{i}_1929_genes_annotated.txt"
    output_file = f"{path}/synonymous_rare/chr{i}_total_presumable_plps.txt"
    
    df = pd.read_csv(input_file, low_memory=False,
                     sep='\t')
    df_syn = df[
        df['synonymous'] & 
        (df['hets'] <= 20) &
        (df['homs'] <= 0)  
    ]
        
    df_syn.to_csv(output_file, sep='\t', index=False)

print ("Synonymous data filtered!", flush=True)

result = []
for i in tqdm.tqdm(range (1, 23)):
    plp_file = f"{path}/synonymous_rare/chr{i}_total_presumable_plps.txt"
    
    result.append(pd.read_csv(plp_file, sep='\t'))

result = pd.concat(result)
result.sort_values(by=['chr', 'position'])

result.to_csv(f"{path}/synonymous_rare/all_chr_total_presumable_plps_HFE_final_sorted.txt", sep='\t')

In [ ]:
!wc -l .../450k/plp_selection/synonymous_rare/all_chr_total_presumable_plps_HFE_final_sorted.txt

# Filter synonymous variants in cohort data

In [ ]:
# original files with samples variants generated in RAP per chromosome
rap_files = glob.glob(".../450k/RAP_output_per_chr/*.all_parts_final.csv")

## file, that contains all PLPs found in the cohort
all_plps_file = (
    ".../450k/plp_selection/synonymous_rare/all_chr_total_presumable_plps_HFE_final_sorted.txt"
)

# output folder for PLPs
## bugfix with gnomad frequency -- now in %
output_folder= '.../450k/RAP_output_per_chr/filtered_plps/synonymous_rare'


# create output folder if not exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
# filter out non-PLP variants for every chromosome

VariantFeatures().filter_plps_in_samples(
    rap_files = rap_files, 
    output_folder = output_folder, 
    all_plps_file = all_plps_file
)

# Sample synonymous 

In [ ]:
plps = (".../450k/plp_selection/basic/new_gene_names/"
        "new_freq/new_relatedness/all_chr_total_presumable_plps_HFE_final_sorted.txt"
)
plps = pd.read_csv(plps, sep='\t')
plps = plps[(plps['hets'] <= 20) & (plps['homs'] <= 0)]

print ("Number of PLPs for regressions:", plps.shape[0])

syns = (".../450k/plp_selection/synonymous_rare/"
        "all_chr_total_presumable_plps_HFE_final_sorted.txt"
)
syns = pd.read_csv(syns, sep='\t')

In [ ]:
for i in range(1000):
        syns.sample(n=plps.shape[0]).to_csv(f".../450k/plp_selection/synonymous_rare/"
                f"all_chr_total_presumable_plps_HFE_final_sorted_samples/sample_{i}.csv", sep='\t')